БИБЛИОТЕКИ

In [1]:
import pandas as pd
import numpy as np

In [2]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ИМЕНА ФАЙЛОВ ВХОДА/ВЫХОДА

In [3]:
# Входной обучающий датасет
train_dataset_file = '/content/drive/My Drive/Bases/DTP/train_dataset_train.csv'

# Входной тестовый датасет
test_dataset_file = '/content/drive/My Drive/Bases/DTP/test_dataset_test.csv'

# Выходной файл тестовой выборки
x_test_file = '/content/drive/My Drive/Bases/DTP/x_test.dat'

# Выходные файлы обучающей выборки для Шага №1
x_train_0_file = '/content/drive/My Drive/Bases/DTP/x_train_0.dat'
y_train_0_file = '/content/drive/My Drive/Bases/DTP/y_train_0.dat'

# Выходные файлы обучающей выборки для Шага №2
x_train_1_file = '/content/drive/My Drive/Bases/DTP/x_train_1.dat'
y_train_1_file = '/content/drive/My Drive/Bases/DTP/y_train_1.dat'

# Выходные файлы обучающей выборки для Шага №3
x_train_2_file = '/content/drive/My Drive/Bases/DTP/x_train_2.dat'
y_train_2_file = '/content/drive/My Drive/Bases/DTP/y_train_2.dat'


ЗАГРУЗКА ДАТАСЕТОВ И ОБЪЕДИНЕНИЕ ИХ В ЕДИНУЮ ТАБЛИЦУ

In [4]:
# загружаем обучающий датасет...
df = pd.read_csv(train_dataset_file)

# фиксируем размер обучающего датасета 
train_base_size = df.shape[0]

# загружаем тестовый датасет...
df_test = pd.read_csv(test_dataset_file)

# объединяем датасеты в единую таблицу
for i in range(df_test.shape[0]):
  rab = {}
  rab['id'] = df_test['id'][i]
  rab['Дата'] = df_test['Дата'][i]
  rab['Время'] = df_test['Время'][i]
  rab['Место'] = df_test['Место'][i]
  rab['Улица'] = df_test['Улица'][i]
  rab['Дом'] = df_test['Дом'][i]
  rab['Дорога'] = df_test['Дорога'][i]
  rab['Километр'] = df_test['Километр'][i]
  rab['Метр'] = df_test['Метр'][i]
  rab['Вид ДТП'] = df_test['Вид ДТП'][i]
  rab['Погибло'] = 0
  rab['Погибло детей'] = 0
  rab['Ранено'] = 0
  rab['Ранено детей'] = 0
  df = df.append(rab, ignore_index=True)

ФУНКЦИИ ДЛЯ РАЗБИВКИ СТОЛБЦА 'МЕСТО' НА ТРИ ЧАСТИ

In [5]:
# извлечение первой составляющей из столбца 'Место' 
def get_place_a(val):

  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  pos = rab.find(',')
  if pos > -1:
    rab = rab[:pos]

  return rab

# извлечение второй составляющей из столбца ('Место') 
def get_place_b(val):
  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  pos = rab.find(',')
  if pos > -1:
    rab = rab[:pos]

  return rab  

# извлечение третьей составляющей из столбца ('Место') 
def get_place_c(val):
  rab = val
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
  
  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]

  pos = rab.find(',')
  if pos == -1:
    return '0'
  else:
    rab = rab[pos+2:]
 
  return rab   

ОБРАБОТКА ТАБЛИЦЫ

In [6]:
# пропуски заполняем 0
df = df.fillna('0') 

# извлекаем месяц
df['Месяц'] = df['Дата'].str[3:5].astype(int)

# извлекаем час...
df['Час'] = df['Время'].str[:2].astype(int)

# удаляем столбцы "Дата" и "Время"
del df['Дата']
del df['Время']

# раскладываем место на три части...
df['Место_1'] = [get_place_a(x) for x in df['Место']] 
df['Место_2'] = [get_place_b(x) for x in df['Место']] 
df['Место_3'] = [get_place_c(x) for x in df['Место']] 
# удаляем столбец "Место"
del df['Место']

# приводим Метры и Километры в int
df['Километр'] = df['Километр'].astype('int')
df['Метр'] = df['Метр'].astype('int')

# добавляем столбец 'Место_4' (на основе слияния Улица-Дом либо Дорога-Километр-Метр)

val = []

for i in range(df.shape[0]):

  if df['Улица'][i] == '0':
    val.append(df['Дорога'][i] + df['Километр'][i].astype('str'))
  else:
    val.append(df['Улица'][i] + df['Дом'][i])

df['Место_4'] = val

# Переводим нужные столбцы в категории

df["Вид ДТП"] = pd.Categorical(df["Вид ДТП"])
df["Вид ДТП"].astype('category').cat.codes
df["Вид ДТП"] = df["Вид ДТП"].cat.codes

df["Место_1"] = pd.Categorical(df["Место_1"])
df["Место_1"].astype('category').cat.codes
df["Место_1"] = df["Место_1"].cat.codes

df["Место_2"] = pd.Categorical(df["Место_2"])
df["Место_2"].astype('category').cat.codes
df["Место_2"] = df["Место_2"].cat.codes

df["Место_3"] = pd.Categorical(df["Место_3"])
df["Место_3"].astype('category').cat.codes
df["Место_3"] = df["Место_3"].cat.codes

df["Место_4"] = pd.Categorical(df["Место_4"])
df["Место_4"].astype('category').cat.codes
df["Место_4"] = df["Место_4"].cat.codes

df["Месяц"] = pd.Categorical(df["Месяц"])
df["Месяц"].astype('category').cat.codes
df["Месяц"] = df["Месяц"].cat.codes

df["Час"] = pd.Categorical(df["Час"])
df["Час"].astype('category').cat.codes
df["Час"] = df["Час"].cat.codes

# удаляем ненужные столбцы
del df['Улица'], df['Дом'], df['Дорога'], df['Километр'], df['Метр']

In [7]:
df

,id,Вид ДТП,Погибло,Погибло детей,Ранено,Ранено детей,Месяц,Час,Место_1,Место_2,Место_3,Место_4
0,490103984,16,0,0,0,0,6,17,4,0,0,1036
1,490097169,16,0,0,0,0,4,17,4,0,0,1003
2,490031781,0,0,0,0,0,0,14,4,0,0,6739
3,490021246,0,0,0,0,0,4,11,12,98,116,1552
4,490041585,2,0,0,0,0,0,1,13,0,0,4027
...,...,...,...,...,...,...,...,...,...,...,...,...
42554,490014641,0,0,0,0,0,6,23,2,9,0,2610
42555,490033115,0,0,0,0,0,0,15,13,0,0,4036
42556,490008221,0,0,0,0,0,8,20,12,0,0,177
42557,490119585,16,0,0,0,0,10,16,13,72,0,4287


ДОБАВЛЯЕМ СТОЛБЕЦ 'КЛАСС' С СИМВОЛАМИ В СООТВЕТСВИИ С ПРОИСШЕСТВИЯМИ

In [8]:
df['Класс'] = (df['Погибло'].astype('str') + 
               df['Погибло детей'].astype('str') + 
               df['Ранено'].astype('str') + 
               df['Ранено детей'].astype('str'))

# смотрим на распределение данных по классам
df['Класс'][df.index < train_base_size].value_counts()

0000    30993
0010     2918
0020      563
0011      506
1000      211
0030      141
0021       91
0022       43
0031       38
1010       38
0040       30
0041       27
1020       24
0032       19
2000       18
0050       17
0042       15
1030        7
0033        7
3000        6
2010        6
0052        4
3061        3
0084        3
0062        3
1110        3
2070        3
0060        3
2020        3
2120        3
0054        3
1031        2
1041        2
1040        2
2030        2
0051        2
3110        2
1021        2
1051        2
1022        2
1121        2
1100        1
1131        1
Name: Класс, dtype: int64

ФОРМИРОВАНИЕ И СОХРАНЕНИЕ ТЕСТОВОЙ ВЫБОРКИ

In [ ]:
x_samples = []

for i in range(train_base_size, df.shape[0]):

    x_vec = []
  
    x_vec.append(df['Вид ДТП'][i].astype('float')) 

    x_vec.append(df['Место_1'][i].astype('float')) 
    
    x_vec.append(df['Место_2'][i].astype('float')) 
    
    x_vec.append(df['Место_3'][i].astype('float')) 

    x_vec.append(df['Место_4'][i].astype('float')) 
    
    x_vec.append(df['Месяц'][i].astype('float')) 

    x_vec.append(df['Час'][i].astype('float')) 

    x_samples.append(x_vec)
    
x_test = np.array(x_samples)

x_test.tofile(x_test_file)

print('Тестовая выборка сохранена в файл:', x_test_file)
print('Размерность массива тестовой выборки: ', x_test.shape)

Тестовая выборка сохранена в файл: /content/drive/My Drive/Bases/DTP/x_test.dat
Размерность массива тестовой выборки:  (6788, 7)


ФОРМИРОВАНИЕ ОБУЧАЮЩЕЙ ВЫБОРКИ ДЛЯ БИНАРНОГО ОТСЕЧЕНИЯ КЛАССА '0000' (ШАГ 1)

In [ ]:
x_samples = []
y_samples = []

for i in range(train_base_size):

  x_vec = []
  
  x_vec.append(df['Вид ДТП'][i].astype('float')) 

  x_vec.append(df['Место_1'][i].astype('float')) 
    
  x_vec.append(df['Место_2'][i].astype('float')) 

  x_vec.append(df['Место_3'][i].astype('float')) 

  x_vec.append(df['Место_4'][i].astype('float')) 
    
  x_vec.append(df['Месяц'][i].astype('float')) 

  x_vec.append(df['Час'][i].astype('float')) 

  x_samples.append(x_vec)

  if df['Класс'][i] == '0000':
    
    y_samples.append(0.0)
  
  else:
  
    y_samples.append(1.0)
      
x_train = np.array(x_samples)
y_train = np.array(y_samples)

x_train.tofile(x_train_0_file)
y_train.tofile(y_train_0_file)

print('Обучающая выборка для Шага 1 сохранена в файлы:')
print(x_train_0_file)
print(y_train_0_file)

print('\nРазмерность обучающей выборки для Шага 1: ')
print(x_train.shape)  
print(y_train.shape)  

Обучающая выборка для Шага 1 сохранена в файлы:
/content/drive/My Drive/Bases/DTP/x_train_0.dat
/content/drive/My Drive/Bases/DTP/y_train_0.dat

Размерность обучающей выборки для Шага 1: 
(35771, 7)
(35771,)


ФОРМИРОВАНИЕ ОБУЧАЮЩЕЙ ВЫБОРКИ ДЛЯ БИНАРНОГО ОТСЕЧЕНИЯ КЛАССА '0010' (ШАГ 2)

In [ ]:
x_samples = []
y_samples = []

for i in range(train_base_size):

  if df['Класс'][i] != '0000': # пропуск данных класса '0000'

    x_vec = []
  
    x_vec.append(df['Вид ДТП'][i].astype('float')) 

    x_vec.append(df['Место_1'][i].astype('float')) 
    
    x_vec.append(df['Место_2'][i].astype('float')) 

    x_vec.append(df['Место_3'][i].astype('float')) 

    x_vec.append(df['Место_4'][i].astype('float')) 
    
    x_vec.append(df['Месяц'][i].astype('float')) 

    x_vec.append(df['Час'][i].astype('float')) 

    x_samples.append(x_vec)

    if df['Класс'][i] == '0010':
    
      y_samples.append(0.0)
  
    else:
  
      y_samples.append(1.0)
      
x_train = np.array(x_samples)
y_train = np.array(y_samples)

x_train.tofile(x_train_1_file)
y_train.tofile(y_train_1_file)

print('Обучающая выборка для Шага 2 сохранена в файлы:')
print(x_train_1_file)
print(y_train_1_file)

print('\nРазмерность обучающей выборки для Шага 2: ')
print(x_train.shape)  
print(y_train.shape)  

Обучающая выборка для Шага 2 сохранена в файлы:
/content/drive/My Drive/Bases/DTP/x_train_1.dat
/content/drive/My Drive/Bases/DTP/y_train_1.dat

Размерность обучающей выборки для Шага 2: 
(4778, 7)
(4778,)


ФОРМИРОВАНИЕ ОБУЧАЮЩЕЙ ВЫБОРКИ ДЛЯ ОСТАВШИХСЯ ДАННЫХ (ШАГ 3)

In [ ]:
x_samples = []
y_samples = []

for i in range(train_base_size):

  if df['Класс'][i] != '0000' and df['Класс'][i] != '0010':

    x_vec = []
  
    x_vec.append(df['Вид ДТП'][i].astype('float')) 

    x_vec.append(df['Место_1'][i].astype('float')) 
   
    x_vec.append(df['Место_2'][i].astype('float')) 

    x_vec.append(df['Место_3'][i].astype('float')) 

    x_vec.append(df['Место_4'][i].astype('float')) 
    
    x_vec.append(df['Месяц'][i].astype('float')) 

    x_vec.append(df['Час'][i].astype('float')) 

    x_samples.append(x_vec)

    y_samples.append([df['Погибло'][i].astype('float'),
                      df['Погибло детей'][i],
                      df['Ранено'][i].astype('float'),
                      df['Ранено детей'][i].astype('float')])
      
x_samples = np.array(x_samples)
y_samples = np.array(y_samples)

x_train = np.array(x_samples)
y_train = np.array(y_samples)

x_train.tofile(x_train_2_file)
y_train.tofile(y_train_2_file)

print('Обучающая выборка для Шага 3 сохранена в файлы:')
print(x_train_2_file)
print(y_train_2_file)

print('\nРазмерность обучающей выборки для Шага 3: ')
print(x_train.shape)  
print(y_train.shape)  

Обучающая выборка для Шага 3 сохранена в файлы:
/content/drive/My Drive/Bases/DTP/x_train_2.dat
/content/drive/My Drive/Bases/DTP/y_train_2.dat

Размерность обучающей выборки для Шага 3: 
(1860, 7)
(1860, 4)
